## SquamataAssemballyAMT - Jupyter notebook for batch releasing Audio Magnetotellic (AMT) data to ScienceBase

This module performs the following operations:
- Create list of data directories.
- Identify files accompanying data release.
- Create file listing for metadata XML markup.
- Identify and load MT EDI file.
- Clean up and reformat harvested values to be XML metadata complient.
- Create User Editable Keywords Listing
- Create entity and attribute XML markup.
- Poplulate metadata template
- Validate metadata; create error log; create HTML and FGDC Text versions of the metadata. (In development - use https://mrdata.usgs.gov/validation/ for validation in the interim)

Known issues needing repair:
- Fix procstep section; do we need this function to collect file information or do we plan on handeling this with boilerplate.

## Future development plans for SquamataSB

- Create all child metadata files from first example created in previous steps. 
- Batch upload files to ScienceBase.
- Batch remove files from ScienceBase. 
- Change ScienceBase parameters such as citation information, add orcid ids, add USGS CMS tags, etc. 

### Instructions
- Create a template xml format that contains boilerplate text common to all childeren in a data release.  Be sure this template contains the approriate curly bracket tags, {SquamataTagExample} used to populate the template using SquamataAMT.

### To execute a function/command select a cell and Hold-Shift + Press-Enter

**The 'r' signifies a string literal. Use for paths.**

Metadata wizard:  Advanced, Open In a jupyter Notebook?
Metadata Wizard 2.o from ScienceBase

v1.1 
- created stack loop for batch mode processing
- harvest process dates
- seperatate out different text fils for A+E, cross power, fourier, etc.
- correct boiler plate for A+E and process steps

v1.2
- fix procedure date harvest bug, typo in first print statement

v1.3
- fix procedure date harvest to harvest file modified date instaed of file created date using "m" time instaed of "c" time; e.g. we want, time.ctime(os.path.getmtime(avgList[0])) NOT time.ctime(os.path.getctime(avgList[0]))
- Fix attribute description boiler plate for BP and SD text files.
- Add entity and attributes for edi file
- add entity and attributes for BP and SD files
- correct loops to create entity and attributes for all FC and TC files, not just the last one in the loop.

v1.4
 - fix attribute description boiler plate for BP.txt and SD.txt (removed the word 'binary')
 - fix attribute description boiler plate for RSP files (BF's units are Volts/Gamma, EF's units are Volts)
 - fix attribute description boiler plate for FC files (replaced word 'Time Series' with 'Fourier Coefficient')
 - fix attribute description boiler plate for BP files (replaced word 'Time Series' with 'Cross Power')
 
v1.5
 - assign the edi file name to variable 'EdiFileName' so it is correctly harvested by the XML template {EdiFileName} tag. 
 - Add code to harvest TS1 file modified time,  {CollectionDate} for "Acquire multiple high frequency time series files TS1 file extension)..." process step. 

v1.6
 - fix xml tagging errors in the procstep section of the metadata
 - fix indexing so that it pulls in all 31 values for the entity and attribute calculations of the EDI file
 
v1.7
 - fix xml definition and unit errors in the Make Array attributes for impedances
 - fix xml definition and unit errors in the Make Array attributes for tipper parameters
 
v1.8
 - correct computed parameter definition for both the impedance VAR and the tipper VAR attributes so that search on "R" in file name does not pick up .VAR files.
 - cleaned up formatting issues so that the correct number of tabs appear in the final xml output between tags
 
v1.9
 - Correct Tipper varience attribute output for var.exp.
 - Correct pandas data frame problem regarding min/max values for computed MT parameters
 
v1.10
 - revise MT-MetadataSilvertonTemplate-v1-10 to reconcile MRJ's review of Silverton AMT data
 - correct Create editable keywords to reconcile MRJ's review of Silverton AMT data 
 - correct missing Tipper TXR.EXP and TYR.EXP attributes
 - correct assign variables for xml file find and replace to include range of each COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters
 - correct Create the Entity and Attributes for the .txt files from detailed to overview
 - correct Create the Entity and Attributes for the FC listing from detailed to overview
 - correct Create the Entity and Attributes for the Time series .TS? listing from detailed to overview
 - correct Create the Entity and Attributes for the .BP? listing from detailed to overview
 - correct SD files Entity and Attributes from detailed to overview
 - add variables for Replace values of current metadata template with the appropriate values  to include range of all COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters
 - need to fix Import entity and attributes - !****COMPUTED PARAMETERS****! to include range of all COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters
 
v1.11
 - fixed Import entity and attributes - !****COMPUTED PARAMETERS****! to include range of all COH, EPREDCOH, SIGAMP, and SIGNOISE computed parameters.
 
v1.12
 - revise and sort themekeys to obey Stu Giles review of Arkansas AMT metadate review.
 
v1.13
 - Brian fixed errors with entity and attributes.
 
v1.14
 - corrected spelling of "sychronous" to "synchronous" in Entity and Attribute Overview
 - way to add URLs for overview section??? - just added a variable for now
 - correct the number of sig figs for bounding coordinates.  Truncate or round.  Check In [14] versus In [17]
 - corrected Vertical Positional Accuracy Report: "Elevations were determined from USGS, The National Map, Bulk Point Query Service based" to "Elevations were determined from The USGS National Map Bulk Point Query Service based"
 - corrected min max value reversal for 'FREQ' in the template
 - corrected wrong value being read in for Phx Max being read in as "Low F'

In [1]:
# Phil Brown (pbrown@usgs.gov) 2019 Beta
# Working Python 3 Notebook used to facilitate the release of Audio Magnetotelluric (AMT) Data to ScienceBase.

In [2]:
# Test Cell
print ("Jupyter is working.") #To run this cell, hold down Shift and press Enter.

Jupyter is working.


In [3]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

C:\ProgramData\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


# 1) Step One - Set Directory Paths
## Please set directory paths below
### Directory paths include
- Data Path: This is the path to the data, data structure should have a directory for each station
- Template Path: The path to the XML metadata template being used for the data.  This template should already include all information common to all child metadata files e.g. originators, larger work citation, etc.

In [4]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
mtDataPath = r"C:\CurrentWork\DataReleases\patagoniaAZ\Data" #The 'r' signifies a string literal. Use for paths.
mtMetaDataTemplatePath = r"C:\CurrentWork\DataReleases\patagoniaAZ"
mtMetaDataTemplateName = "AMT-MetadataArizTemplatev1-14PhilFinal4120.xml"

In [5]:
#Check Paths for the fun of it
print ('The MT Data Path is: ' + '"' + mtDataPath + '"')
mtMetaDataTemplatePath + "\\" + mtMetaDataTemplateName

The MT Data Path is: "C:\CurrentWork\DataReleases\patagoniaAZ\Data"


'C:\\CurrentWork\\DataReleases\\patagoniaAZ\\AMT-MetadataArizTemplatev1-14PhilFinal4120.xml'

## Now, let's explore our data. 
- What files do we have? 
- What files do we import values from?

In [6]:
#Explore data files and directory structure hosted below the provided provided parent data directory

In [7]:
#Produce directory listing of station (SB Object Children)
#Either set up the root directory with station subdirectories only or delete non-station directories from the list array
#mtDataDirList = os.walk(mtDataPath)
#mtDataDirList = [entry.path for entry in os.scandir(mtDataPath) if entry.is_dir()]
mtDataDirList = next(os.walk(mtDataPath))[1]
mtDataDirList

['AMT01', 'AMT03', 'AMT04', 'AMT05', 'AMT06', 'AMT07']

# We can select an indivdual station to test or run code for all stations in batch mode.  

In [307]:
#!!!!!!!!!Run this code and select A single station !!!!!!!!!
#!!!!!!!!!Skip the loop cell below and run cells sequentually !!!!!!!!!!
#!!!!!!!!!Use Shift-Enter to then execute each cell one by one!!!!!!!!!! 
mtStationPath = mtDataPath + '\\' + mtDataDirList[5]
mtStationPath

'C:\\CurrentWork\\DataReleases\\patagoniaAZ\\Data\\AMT07'

In [308]:
#Insert the child URL Here
childURL = 'https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e'

# ----------------- NEXT STEP IS BROKEN LOOP (SKIP!!!) ----------------------------------

In [ ]:
#!!!!!!!!!!!!!!!!!!  Run this code to loop through ALL Data Sets !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!This code will produce metadata files for ALL stations in batch mode!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#!!!!!!!!!!!!!!!!!!  Skip running this code if processing a single station !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#for i in range (0,len(mtDataDirList)):
#    mtStationPath = mtDataPath + '\\' + mtDataDirList[i]
#    from IPython.display import Javascript
#    display(Javascript('IPython.notebook.execute_cells_below()'))

# -----------------------------------------------------------------------------------------------------------

In [309]:

#Look for EDI file to load
ediList = glob.glob(os.path.join(mtStationPath, '**/*MT*.edi'),  recursive=True)
ediPath = ediList[0]
#ediList
print ('EDI File Path:\n' + ediPath)
ediPathArray = ediPath.split('\\')
ediFile = str(ediPathArray[len(ediPathArray)-1])
print ('EDI File:\n' + ediFile)
EdiFileName = ediFile
#ediPathArray

EDI File Path:
C:\CurrentWork\DataReleases\patagoniaAZ\Data\AMT07\USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.edi
EDI File:
USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.edi


## Harvest from the MT EDI file. 
### Parameters include:
- ProductId=USA-New_Mexico-Rio_Grande_Rift-San_Luis_Basin-2009-AMT01.edi
- ExternalUrl Url=https://doi.org/10.5066/F72F7MQ7
- Attachment Filename=https://pubs.usgs.gov/of/2011/1264/report/OF11-1264.pdf
- Survey Purpose Description: 
- Data Description:
- Citation Title=Audiomagnetotelluric data, Taos Plateau Volcanic Field, New Mexico
- Citation Authors=Chad E. Ailes, Brian D. Rodriguez
- Citation Year=2011
- YearCollected=2009
- Country=USA                                  
- Ellipsoid=Clarke 1866                                                          
- Location datum=NAD27 CONUS                                                     
- SITE LATITUDE=36.752985000                                                     
- SITE LONGITUDE=-105.560966167                                                  
- Elevation units="meters"=2608.00                                                                     
- Start=2009-07-21T19:52:03 UTC/GMT
- End=2009-07-21T20:34:20 UTC/GMT
- ProcessingTimeSeriesUsed:
         wp01A1.bp1                                                                     
         wp01A2.bp1                                                                     
         wp01A1.sd6                                                                     
         wp01A2.sd6                                                                     
         wp01A1.sd7                                                                     
         wp01A2.sd8                                                                     
         wp01A2_3.sd9 
- Entities and Attributes:
    - FREQUENCIES
    - IMPEDANCE ROTATION ANGLES
    - IMPEDANCES
    - TIPPER PARAMETERS
    - COMPUTED PARAMETERS


## Lets now import and index values from the EDI Files
- We need these values for the metadata template.  
- We also want to run stats on some of these values for the entity and attributes section

In [310]:
    #Load EDI File and Read It
    ediFile = open(ediPath, 'r')
    ediContent = ediFile.read()
    ediFile.close()
    print(ediContent)


>HEAD                                                                           
                                                                                
  DATAID=Sunnyside                                                              
  ACQBY=USGS                                                                    
  ACQDATE=2008-05-03
  STATE=Arizona                                                                 
  COUNTY="Santa Cruz"                                                           
  UNITS=M                                                                       
  STDVERS=1.0                                                                   
  PROGVERS=GEOTOOLS_2.3                                                         
  PROGDATE=09/16/94                                                             
                                                                                
>INFO   MAXLINES=1000                                                           
       

In [311]:
#Now assign values to the SB MetaDataWizard Template unknowns
list_ = ediContent.splitlines()
list_length = len (list_)

# there are probally easier ways to loop through the below but I like having it all hard coded upfront
# it's easire to track an change for me
# Use the examples provided below to extract additional parameters
# Not that all variables being collected are not necessarily used in populating the template.
# Note that values can be hardcoded into the metadata xml template and/or harvested from the edi file

for X in list_:
  if "ProductId" in X:
    productArray = X.split('=')
    productIdArray = productArray[1].split('.')
    productId = productIdArray[0]
    # We may want to reformat this are parse out this name further for use with a root name based on the Data Release Title?
    productId = productId.replace("-", " ")
    productId = productId.replace("_", " ")
    drTitle = productId
    print ('Child Title: ' + productId)
  elif "ExternalUrl Url" in X:
    externalURLArray = X.split('=')
    externalURL = externalURLArray[1]
    print ('<onlink>: ' + externalURL)
  elif "STATE" in X:
    stateArray = X.split('=')
    state = stateArray[1].replace('"', "") #remove quotes around state
    print ('State: ' + state)
  elif "COUNTY" in X:
    countyArray = X.split('=')
    county = countyArray[1]
    print ('County: ' + county)
  elif "Start" in X:
    startArray = X.split('=')
    start = startArray[1]
    print ('Start: ' + start)
  elif "End" in X:
    endArray = X.split('=')
    end = endArray[1]
    print ('End: ' + end)
  elif "Attachment Filename" in X and "http" in X:
    lgwrklinkArray = X.split('=')
    lgwrklink = lgwrklinkArray[1]
    print ('Attachment Filename Link: ' + lgwrklink)
  elif "Citation Title" in X:
    citTitArray = X.split('=')
    citTit = citTitArray[1]
    print ('Citation Title: ' + citTit)
  elif "Citation Authors" in X:
    citNamesArray = X.split('=')
    citAuthorsArray = citNamesArray[1].split(',')
    for author in citAuthorsArray:
     author = author.strip()
     print ('Author: '+ author)
  elif "Citation Year" in X:
    citYearArray = X.split('=')
    citYear = citYearArray[1]
    print ('Citation Year: ' + citYear)
  elif "YearCollected" in X:
    yearColArray = X.split('=')
    yearCol = yearColArray[1]
    print ('Year Collected: ' + yearCol)
  elif "Ellipsoid" in X:
    ellipsoidArray = X.split('=')
    ellipsoid = ellipsoidArray[1]
    print ('Ellipsoid: ' + ellipsoid)
  elif "Location datum" in X:
    locDatumArray = X.split('=')
    locDatum = locDatumArray[1]
    print ('Local datum: ' + locDatum)
  elif "SITE LATITUDE" in X:
    sitLatArray = X.split('=')
    sitLat = sitLatArray[1] #may need to trim extra spaces !!!
    sitLat = float(sitLat) #round to correct number of sigfigs - do this above; comment this in if you want to use this function 
    sitLat = round(sitLat, 6)
    print ('Site latitude: ' + str(sitLat))
  elif "SITE LONGITUDE" in X:
    sitLonArray = X.split('=')
    sitLon = sitLonArray[1] # may need to trim extra spaces !!!
    sitLon = float(sitLon) #round to correct number of sigfigs - do this above; comment this in if you want to use this function 
    sitLon = round(sitLon, 6)
    print ('Site longitude: ' + str(sitLon))
  elif "Elevation units" in X:
    elevationStringArray = X.split('=')
    siteElevation = elevationStringArray[2] 
    print ('Site Elevation: ' + siteElevation)
    elevationUnits = elevationStringArray[1].replace('"', "")
    print ('Elevation Units: ' + elevationUnits)
# Code below returns values that occupy more than one line
    
for i in range(list_length):
 value = list_[i] 
 if value.replace(" ", "") == 'SurveyPurposeDescription:':
  startIndPurpose = i + 1
   #print ('startIndPurpose: ' + str(startIndPurpose))
 if value.replace(" ", "") == 'DataDescription:':
   endIndPurpose = i - 1
   #print ('endIndPurpose: ' + str(endIndPurpose))
purpose = list_[startIndPurpose]
for j in range(startIndPurpose + 1,endIndPurpose): 
    purpose = purpose + list_[j]
    purposeClean = re.sub(' +', ' ',purpose)
print ('\nAbstract:\n\t' + purposeClean)

for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == 'DataDescription:':
   startIndDescription = k + 1
   #print ('startIndDescription: ' + str(startIndDescription))
 if value.replace(" ", "") == 'FILECREATOR:':
   endIndDescription = k - 9
   #print ('endIndDescription: ' + str(endIndDescription))
description = list_[startIndDescription]
for l in range(startIndDescription + 1,endIndDescription): 
    description = description + list_[l]
    descriptionClean = re.sub(' +', ' ',description)
print ('\nPurpose:\n\t' + descriptionClean) 


State: Arizona                                                                 
County: "Santa Cruz"                                                           
Child Title: USA Arizona Patagonia Mtns Sunnyside Porphyry 2010 AMT07
<onlink>: https://doi.org/10.5066/P91599D6 
Attachment Filename Link: http://pubs.usgs.gov/of/2010/1311/pdf/OF10-1311.pdf        
Citation Title: Audio-magnetotelluric survey to characterize the Sunnyside porphyry copper system in the Patagonia Mountains, Arizona
Author: Brian D. Rodriguez
Author: Jay A. Sampson
Citation Year: 2010                                                             
Year Collected: 2008
Ellipsoid: Clarke 1866                                                          
Local datum: NAD27 CONUS                                                     
Site latitude: 31.471463
Site longitude: -110.688648
Site Elevation: 1551.00                                               
Elevation Units: meters
Start: 2008-05-03T17:52:50 UTC/GMT
End: 2008-05

In [312]:
#Reformat product ID to more correctly reflect child (station) name
arrChildID = productId.split(' ')
strChildID = arrChildID[-1]
strChildID


'AMT07'

In [313]:
# Now let's format the start time and end time to be what the XML file wants for <begdate> and <enddate>

begdateArr = start.split(' ')
begdate_str = begdateArr[0]
begdate_obj = dateutil.parser.parse(begdate_str)
begdate = begdate_obj.strftime('%Y%m%d')
print('<begdate> ', begdate) 

enddateArr = end.split(' ')
enddate_str = enddateArr[0]
enddate_obj = dateutil.parser.parse(enddate_str)
enddate = enddate_obj.strftime('%Y%m%d')
print('<enddate> ', enddate) 


<begdate>  20080503
<enddate>  20080503


In [314]:
#Now we reformat the lat  and longitude to 6 sig figs as well as trim of any extra spaces
#Brian seems to be stripping out the sig figs now so there is no need.  
#So this strip doesn't seem to work - let's try it below using the round-up function.
sitLat = str(sitLat)
sitLat = sitLat.strip()
sitLon = str(sitLon)
sitLon = sitLon.strip()

print ('Site latitude: ' + sitLat)
print ('Site longitude: ' + sitLon)

Site latitude: 31.471463
Site longitude: -110.688648


In [315]:
# Now Reformat county by trimming the extra spaces
county = county.strip()
# comment out below.  This is because the edi file now has the county in quotes and I don't want to strip those out
county = "Santa Cruz"
print ('County: ' + county)

County: Santa Cruz


In [316]:
## Create editable keywords example.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
keywords = widgets.Textarea(
    value='\t\t<keywords>\n\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>\n\t\t\t\t<themekey>geoscientificInformation</themekey>' \
    + '\n\t\t\t\t<themekey>economy</themekey>\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>None</themekt>' \
    + '\n\t\t\t\t<themekey>Mineral Resources Program</themekey>' \
    + '\n\t\t\t\t<themekey>MRP</themekey>' \
    + '\n\t\t\t\t<themekey>Geology, Geophysics, and Geochemistry Science Center</themekey>' \
    + '\n\t\t\t\t<themekey>GGGSC</themekey>' \
    + '\n\t\t\t\t<themekey>electric field (earth)</themekey>' \
    + '\n\t\t\t\t<themekey>impedance</themekey>\n\t\t\t\t<themekey>tipper</themekey>' \
    + '\n\t\t\t\t<themekey>apparent resistivity</themekey>\n\t\t\t\t<themekey>impedance phase</themekey>' \
    + '\n\t\t\t\t<themekey>impedance strike</themekey>\n\t\t\t\t<themekey>magnetotelluric</themekey>' \
    + '\n\t\t\t\t<themekey>MT</themekey>\n\t\t\t\t<themekey>audiomagnetotelluric</themekey>' \
    + '\n\t\t\t\t<themekey>AMT</themekey>\n\t\t\t\t<themekey>sounding</themekey>' \
    + '\n\t\t\t\t<themekey>Sunnyside porphyry</themekey>\n\t\t\t\t<themekey>copper</themekey>' \
    + '\n\t\t\t</theme>\n\t\t\t<theme>\n\t\t\t\t<themekt>USGS Thesaurus</themekt>' \
    + '\n\t\t\t\t<themekey>magnetic field (earth)</themekey>' \
    + '\n\t\t\t\t<themekey>geophysics</themekey>\n\t\t\t\t<themekey>GPS measurement</themekey>' \
    + '\n\t\t\t\t<themekey>magnetotelluric surveying</themekey>\n\t\t\t</theme>\n\t\t\t<place>' \
    + '\n\t\t\t\t<placekt>USGS Geographic Names Information System (GNIS), https://geonames.usgs.gov</placekt>' \
    + '\n\t\t\t\t<placekey>San Rafael Valley</placekey>' \
    + '\n\t\t\t\t<placekey>Patagonia Mountains</placekey>' \
    + '\n\t\t\t\t<placekey>' + county + ' County</placekey>\n\t\t\t</place>\n\t\t\t<place>' \
    + '\n\t\t\t\t<placekt>Common geographic areas</placekt>\n\t\t\t\t<placekey>Arkansas</placekey>' \
    + '\n\t\t\t</place>\n\t\t\t</keywords>', \
    placeholder='Type something',
    #description='String:',
    layout=Layout(width='100%', height='666px'),
    disabled=False
)
print ('Keywords list created.')

Keywords list created.


### Change the text in the textbox below to relflect what should be included as the key words for all child items

Note that changing the text below at any time creates a keywords section of the metadata seen EXACTLY as it is shown below

In [317]:
# Run this cell for key word text to edit.  
# One can edit the text in place and these changes will be present in the created metadata.  
# When complete move on to the next step

display(keywords)

Textarea(value='\t\t<keywords>\n\t\t\t<theme>\n\t\t\t\t<themekt>ISO 19115 Topic Category</themekt>\n\t\t\t\t<t…

### The module below collects the file date property for use in creating the process description dates

In [318]:
# module to get file process dates for process descriptions
# Visit https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior for time format standards
# Note that get c time gives creation date.  Get m time gives creation date.
# We want cration time e.g. we want, time.ctime(os.path.getmtime(avgList[0])) NOT time.ctime(os.path.getctime(avgList[0]))

# Set {CollectionDate} to the station end date
strCollectionDate = enddate
print ("{CollectionDate} " + strCollectionDate)

# {CombinationDate} Get the Time stamp for the AVG file
# First we need to get the file path
avgList = glob.glob(os.path.join(mtStationPath, '*.AVG'),  recursive=True)
#print (avgList)
avgTime = time.ctime(os.path.getmtime(avgList[0]))
print ("avgTime: " + avgTime)
avgTime_obj = datetime.datetime.strptime(avgTime,'%a %b %d %H:%M:%S %Y')
strCombinationDate = datetime.datetime.strftime(avgTime_obj,'%Y%m%d')
print ("{CombinationDate} " + strCombinationDate)


# {ConversionDate} is the date of the older edi file, this is the second file in the edi list
# No that isn't it, I filtered to get the newer edi file, oh I'll just find the file 
# in an array of edi files that isn't the other one

ediAllList = glob.glob(os.path.join(mtStationPath, '*.edi'),  recursive=True)
#print (ediAllList)
for i in range (0,len(ediAllList)):
  if ediAllList[i] != ediPath:
    OriginalEDIPath = ediAllList[i]
os.path.getctime(OriginalEDIPath)
oediTime = time.ctime(os.path.getmtime(OriginalEDIPath))
print ("oediTime: " + oediTime)
oediTime_obj = datetime.datetime.strptime(oediTime,'%a %b %d %H:%M:%S %Y')
strConversionDate = datetime.datetime.strftime(oediTime_obj,'%Y%m%d')
print ("{ConversionDate} " + strConversionDate)

# {RotationDate} Get the Time stamp for the AVG file
# First we need to get the file path
dmpList = glob.glob(os.path.join(mtStationPath, '*.dmp'),  recursive=True)
#print (dmpList)
dmpTime = time.ctime(os.path.getmtime(dmpList[0]))
print ("dmpTime: " + dmpTime)
dmpTime_obj = datetime.datetime.strptime(dmpTime,'%a %b %d %H:%M:%S %Y')
strRotationDate = datetime.datetime.strftime(dmpTime_obj,'%Y%m%d')
print ("{RotationDate} " + strRotationDate)

#{HarvestDate} = final edi file 
os.path.getctime(ediPath)
filetime = time.ctime(os.path.getmtime(ediPath))
print ("Harvest Time: " + filetime)
filetime_obj = datetime.datetime.strptime(filetime,'%a %b %d %H:%M:%S %Y')
strHarvestDate = datetime.datetime.strftime(filetime_obj,'%Y%m%d')
print ("{HarvestDate} " + strHarvestDate)

{CollectionDate} 20080503
avgTime: Thu Apr  2 12:30:50 2020
{CombinationDate} 20200402
oediTime: Thu Apr  2 12:30:50 2020
{ConversionDate} 20200402
dmpTime: Thu Apr  2 12:30:50 2020
{RotationDate} 20200402
Harvest Time: Thu Apr  2 12:30:50 2020
{HarvestDate} 20200402


Entity and Attribute Values for the EDI file.  List !****FREQUENCIES****!,!****IMPEDANCE ROTATION ANGLES****!,!****IMPEDANCES****!,!****COMPUTED PARAMETERS****!

Here we load the frequencies
>!****FREQUENCIES****!

In [319]:
# Import entity and attributes - !****FREQUENCIES****! plan to break some of these individual chunks into objects/functions

# Get Range of Frequency Values in EDI File
for k in range(list_length):

 value = list_[k] 
 if value.replace(" ", "") == '>!****FREQUENCIES****!':
   startIndFrequencies = k + 2
   print ('startIndFrequencies: ' + str(startIndFrequencies))
 
 if value.replace(" ", "") ==  '>!****IMPEDANCEROTATIONANGLES****!':
   endIndFrequencies = k - 1
   print ('endIndFrequencies: ' + str(endIndFrequencies))

frequencyData = []
fdata = []
fdataTemp = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
frequencyDF = pd.DataFrame(fdata)
for j in range(startIndFrequencies,endIndFrequencies):
    fdataTemp = list_[j]
    fdataTemp = re.sub(' +', ' ',fdataTemp)
    fdataTemp = fdataTemp.split(" ")
    del fdataTemp[0]
    fdata = fdata + fdataTemp
    
print (fdata)  
fdata = np.array(fdata).astype(np.float) #convert String to floats
frequencyDF = pd.DataFrame(fdata,columns=['Frequencies'])
frequencyDF


startIndFrequencies: 275
endIndFrequencies: 282
['2.33700000E+04', '1.95000000E+04', '1.52900000E+04', '1.15900000E+04', '9.00000000E+03', '6.50000000E+03', '4.90000000E+03', '3.55000000E+03', '2.73000000E+03', '2.20000000E+03', '1.87000000E+03', '1.50000000E+03', '1.17000000E+03', '8.85000000E+02', '7.20000000E+02', '5.80000000E+02', '4.60000000E+02', '3.40000000E+02', '2.70000000E+02', '2.10000000E+02', '1.72399994E+02', '1.50000000E+02', '1.22099998E+02', '1.00000000E+02', '9.00000000E+01', '8.59400024E+01', '7.90000000E+01', '6.00600014E+01', '4.15000000E+01', '2.83199997E+01', '1.90400009E+01', '1.22100000E+01', '7.32399988E+00', '4.39400005E+00']


,Frequencies
0,23370.000000
1,19500.000000
2,15290.000000
3,11590.000000
4,9000.000000
5,6500.000000
6,4900.000000
7,3550.000000
8,2730.000000
9,2200.000000


In [320]:
# Now lets get the stats of the frequency data
#Make Array of Max Vallues
frequencyMax = frequencyDF[('Frequencies')].max()
print ('Max. Frequency: ' + str(frequencyMax))
frequencyMin = frequencyDF[('Frequencies')].min()
print ('Min. Frequency: ' + str(frequencyMin))

Max. Frequency: 23370.0
Min. Frequency: 4.39400005


Here we load the Impedance Rotation Angles
>!****IMPEDANCE ROTATION ANGLES****!

In [321]:
# Import entity and attributes - !****IMPEDANCE ROTATION ANGLES****! plan to break some of these individual chunks into objects/functions
# Get Range of IMPEDANCE ROTATION ANGLES in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****IMPEDANCEROTATIONANGLES****!':
   startIndROT = k + 2
   print ('startIndROT: ' + str(startIndROT))
 
 if value.replace(" ", "") ==  '>!****IMPEDANCES****!':
   endIndROT = k - 1
   print ('endIndROT: ' + str(endIndROT))

rdata = []
rdataTemp = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
rotationDF = pd.DataFrame(rdata)
for j in range(startIndROT,endIndROT):
    rdataTemp = list_[j]
    rdataTemp = re.sub(' +', ' ',rdataTemp)
    rdataTemp = rdataTemp.split(" ")
    del rdataTemp[0]
    rdata = rdata + rdataTemp
    
print (rdata)  
rdata = np.array(rdata).astype(np.float) #convert String to floats
rotationDF = pd.DataFrame(rdata,columns=['ZROT'])
rotationDF

startIndROT: 285
endIndROT: 292
['0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00', '0.00000000E+00']


,ZROT
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


In [322]:
# Now lets get the stats of the rotation data

#Make Array of Max Values
rotationMax = rotationDF[('ZROT')].max()
print ('Max. ZROT: ' + str(rotationMax))

#Make Array of Min Values
rotationMin = rotationDF[('ZROT')].min()
print ('Min. ZROT: ' + str(rotationMin))

Max. ZROT: 0.0
Min. ZROT: 0.0


Here we load the impedances
>!****IMPEDANCES****!

In [323]:
# Import entity and attributes - !****IMPEDANCES****! plan to break some of these individual chunks into objects/functions
# Get Range of Impedance Values in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****IMPEDANCES****!':
   startIndImpedances = k + 1
   print ('startIndImpedances: ' + str(startIndImpedances))
 
 if value.replace(" ", "") ==  '>!****TIPPERPARAMETERS****!':
   endIndImpedances = k - 1 
   print ('endIndImpedances: ' + str(endIndImpedances))

#Construct Array of Channel Headers   
count = 0
impedanceLabel = []
impedanceData = []
data = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
impedanceDF = pd.DataFrame(data)
for l in range (startIndImpedances,endIndImpedances): 
    if list_[l][0] == '>':
     temp = list_[l].split(" ", 1)
     #print ('List: ' + list_[l][7])
     impedanceLabel.append((temp[0].split(">"))[1])
     dataTemp = list_[l+1]
     #print ('datatemp: ' + dataTemp)
     for j in range(l+2,l+9):
      dataTemp = dataTemp + list_[j]
      #print ('datatemp: ' + dataTemp)
      dataTemp = re.sub(' +', ' ',dataTemp)
     data = dataTemp.split(" ")
     del data[0] # need to check for empty strings and delete these from the array of the string can't be converted to a float
     del data[len(data)-1] # need to check for empty strings and delete
     #print (data)
     data = np.array(data).astype(np.float) #convert String to floats
     se = pd.Series(data)
     print ((temp[0].split(">"))[1])   
     impedanceDF[((temp[0].split(">"))[1])] = se.values
    
    count = count + 1

#impedanceDF = pd.DataFrame(data, columns=(impedanceLabel))
impedanceDF
#data
#se 

startIndImpedances: 294
endIndImpedances: 401
ZXXR
ZXXI
ZXX.VAR
ZXYR
ZXYI
ZXY.VAR
ZYXR
ZYXI
ZYX.VAR
ZYYR
ZYYI
ZYY.VAR


,ZXXR,ZXXI,ZXX.VAR,ZXYR,ZXYI,ZXY.VAR,ZYXR,ZYXI,ZYX.VAR,ZYYR,ZYYI,ZYY.VAR
0,-72.481041,-26.895327,546.203796,-209.750229,-581.702576,42999.539100,305.648895,544.567993,23786.398400,4789.124510,-2517.046630,1.872569e+06
1,-10.187765,-74.704926,288.127319,-362.640381,291.400024,21166.355500,568.518799,684.682678,14234.731400,-637.449646,-5555.896000,1.045709e+06
2,9.593272,-73.810074,277.981720,-323.308716,567.153687,20519.843800,467.648560,274.558990,4213.118650,-1942.809810,-3245.904790,3.110008e+05
3,-0.299467,-7.919363,31.428133,-153.643860,83.464256,2737.305660,263.855377,-29.802261,1447.287600,-1600.910520,-1091.318360,1.260549e+05
4,-3.642603,-8.922458,18.480774,-32.985550,113.785080,1720.696410,145.166229,-133.471970,645.821045,-1237.876590,48.889843,6.013070e+04
5,4.581573,-8.473061,10.406904,-75.103058,129.943085,971.359253,42.409973,-105.456741,421.976898,-1067.167850,-195.153519,3.938647e+04
6,0.164082,2.401368,30.659172,-62.902290,-50.039639,2867.536130,-51.721188,-159.912231,1057.150020,-258.916443,624.876465,9.887467e+04
7,0.600588,-7.215170,14.716048,-39.525505,49.961697,1334.715090,-94.350960,-109.440628,643.424316,-30.280737,369.973083,5.835725e+04
8,-2.634809,-2.470671,16.165415,-36.793350,17.837036,1680.583250,-102.914749,-85.479759,591.629639,-282.490021,265.139069,6.150680e+04
9,-1.604172,-10.705101,18.072020,-14.589771,89.434143,1726.313840,-79.211891,-85.257698,939.911011,-338.048248,387.860657,8.978417e+04


In [324]:
# Now lets get the stats of the impedance data

# Make Array of Max Values
impedanceMax = []
for i in range (0,len(impedanceLabel)):
    impedanceMax.append(impedanceDF[(impedanceLabel[i])].max())
print ('Impedance Max: ' + str(impedanceMax))

# Make Array of Min Values
impedanceMin = []
for i in range (0,len(impedanceLabel)):
    impedanceMin.append(impedanceDF[(impedanceLabel[i])].min())
print ('Impedance Min: ' + str(impedanceMin))

Impedance Max: [9.59327221, 16.0248756, 546.203796, 79.2853088, 567.153687, 42999.5391, 568.518799, 684.682678, 23786.3984, 4789.12451, 624.876465, 1872568.75]
Impedance Min: [-72.481041, -74.7049255, 0.0715052113, -362.640381, -581.702576, 5.47305202, -125.880112, -159.912231, 0.418522984, -1942.80981, -5555.896, 36.6498032]


In [325]:
#Make Array attributes for impedances
impAtt = ''
for i in range (0,len(impedanceLabel)):
     headerLabel = impedanceLabel[i]
     print ('Header Label: ' + headerLabel + '-' + headerLabel[:3])
     if headerLabel.find('R') != -1 and headerLabel.find('.VAR') == -1:
        impAtti = '\n\t\t<attr>\n\t\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Real component of impedance ' + headerLabel[:3] + '</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t\t<rdommin>'+ str(impedanceMin[i]) \
        + '</rdommin>\n\t\t\t\t\t<rdommax>' + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t\t'\
        + '<attrunit>ohms</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\t\t</attr>'
        impAtt = impAtt + impAtti
     if headerLabel.find('I') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Imaginary component of impedance ' + headerLabel[:3] + '</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t\t<rdommin>'+ str(impedanceMin[i]) + '</rdommin>\n\t\t\t\t\t<rdommax>' \
        + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t\t'\
        + '<attrunit>ohms</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\t\t</attr>'
        impAtt = impAtt + impAtti
     if headerLabel.find('.VAR') != -1:
        impAtti = '\n\t\t<attr>\n\t\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t\t<attrdef>Total variance of impedance of ' + headerLabel[:3] \
        + '</attrdef>' \
        + '\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t\t<attrdomv>\n\t\t\t\t\t<rdom>\n\t\t\t\t\t<rdommin>'+ str(impedanceMin[i]) + '</rdommin>\n\t\t\t\t\t<rdommax>' \
        + str(impedanceMax[i]) + '</rdommax>\n\t\t\t\t\t'\
        + '<attrunit>ohms*ohms</attrunit>\n\t\t\t\t</rdom>\n\t\t\t\t</attrdomv>\n' \
        + '\n\t\t</attr>'
        impAtt = impAtt + impAtti
   
        
print (impAtt)
ImpedanceAttributes = impAtt

Header Label: ZXXR-ZXX
Header Label: ZXXI-ZXX
Header Label: ZXX.VAR-ZXX
Header Label: ZXYR-ZXY
Header Label: ZXYI-ZXY
Header Label: ZXY.VAR-ZXY
Header Label: ZYXR-ZYX
Header Label: ZYXI-ZYX
Header Label: ZYX.VAR-ZYX
Header Label: ZYYR-ZYY
Header Label: ZYYI-ZYY
Header Label: ZYY.VAR-ZYY

		<attr>
				<attrlabl>ZXXR</attrlabl>
				<attrdef>Real component of impedance ZXX</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
				<rdom>
					<rdommin>-72.481041</rdommin>
					<rdommax>9.59327221</rdommax>
					<attrunit>ohms</attrunit>
				</rdom>
				</attrdomv>
		</attr>
		<attr>
			<attrlabl>ZXXI</attrlabl>
				<attrdef>Imaginary component of impedance ZXX</attrdef>
				<attrdefs>U.S. Geological Survey</attrdefs>
				<attrdomv>
				<rdom>
					<rdommin>-74.7049255</rdommin>
					<rdommax>16.0248756</rdommax>
					<attrunit>ohms</attrunit>
				</rdom>
				</attrdomv>
		</attr>
		<attr>
				<attrlabl>ZXX.VAR</attrlabl>
				<attrdef>Total variance of impedance of ZXX</a

Here we load the tipper parameters
>!****TIPPER PARAMETERS****!

In [326]:
# Import entity and attributes - !****TIPPER PARAMETERS****! plan to break some of these individual chunks into objects/functions
# Probably will need two functions for this - one for a single list and one for the long lists with more than one column

# Get Range of TIPPER PARAMETERS in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****TIPPERPARAMETERS****!':
   startIndTipper = k + 1
   print ('startIndTipper: ' + str(startIndTipper))
 
 if value.replace(" ", "") ==  '>!****COMPUTEDPARAMETERS****!':
   endIndTipper = k - 1
   print ('endIndTipper: ' + str(endIndTipper))

#Construct Array of Channel Headers   
count = 0
tipperLabel = []
tipperData = []
tdata = []
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
tipperDF = pd.DataFrame(tdata)
for l in range(startIndTipper,endIndTipper): 
    if list_[l][0] == '>':
     ttemp = list_[l].split(" ", 1)
     #print (ttemp)
     tipperLabel.append((ttemp[0].split(">"))[1])
     tdataTemp = list_[l+1]
     for j in range(l+2,l+9):
      tdataTemp = tdataTemp + list_[j]
      tdataTemp = re.sub(' +', ' ',tdataTemp)
      tdata = tdataTemp.split(" ")
     #print (tdata)
     del tdata[0]
     del tdata[len(tdata)-1] # need to check for empty strings and delete
     tdata = np.array(tdata).astype(np.float) #convert String to floats
     te = pd.Series(tdata)
     print ((ttemp[0].split(">"))[1])   
     tipperDF[((ttemp[0].split(">"))[1])] = te.values
    
    count = count + 1

#tipperDF = pd.DataFrame(tdata, columns=(tipperLabel))
tipperDF
#tdata
#te 

startIndTipper: 403
endIndTipper: 456
TXR.EXP
TXI.EXP
TXVAR.EXP
TYR.EXP
TYI.EXP
TYVAR.EXP


,TXR.EXP,TXI.EXP,TXVAR.EXP,TYR.EXP,TYI.EXP,TYVAR.EXP
0,-0.315614,0.266465,0.033317,0.326460,-2.571306,2.622860
1,-0.205076,0.500829,0.145082,-0.796345,-4.373968,10.658021
2,-0.226825,0.442275,0.103471,-0.709678,-3.828878,7.637955
3,-0.292449,0.495681,0.100157,-0.411795,-4.825277,8.723443
4,-0.236779,0.374639,0.051050,-1.099915,-3.520028,4.753093
5,-0.307058,0.312359,0.026886,0.571984,-3.320099,2.509509
6,-0.102176,0.240700,0.037884,-1.869588,-2.276003,3.543285
7,-0.221427,0.219079,0.026420,-0.938638,-2.153137,2.396209
8,-0.153477,0.162043,0.015438,-0.786318,-1.625795,1.604937
9,-0.230729,0.235241,0.028376,-0.812439,-2.255649,2.710551


In [327]:
# Now lets get the stats of the tipper data

# Make Array of Max Values
tipperMax = []
for i in range (0,len(tipperLabel)):
    tipperMax.append(tipperDF[(tipperLabel[i])].max())
print ('Tipper Max: ' + str(tipperMax))    

# Make Array of Min Values
tipperMin = []
for i in range (0,len(tipperLabel)):
    tipperMin.append(tipperDF[(tipperLabel[i])].min())
print ('Tipper Min: ' + str(tipperMin))

Tipper Max: [-0.102175966, 0.671996057, 0.183176562, 3.60479975, 0.854166269, 10.658021]
Tipper Min: [-0.710182965, -0.362285525, 0.00059370324, -1.86958766, -4.82527685, 0.0516657978]


In [328]:
#Make Array attributes for tipper parameters
tipAtt = ''
for i in range (0,len(tipperLabel)):
     headerLabel = tipperLabel[i]
     print ('Header Label: ' + headerLabel + '-' + headerLabel[:3])
#     if headerLabel.find('R') != -1 and headerLabel.find('.VAR') == -1 and headerLabel.find('.EXP') == -1:
     if headerLabel.find('R') != -1 and headerLabel.find('VAR') == -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Real part of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('I') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Imaginary part of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('.VAR') != -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Total variance of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
     if headerLabel.find('VAR') != -1 and headerLabel.find('.VAR') == -1:
        tipAtti = '\t\t<attr>\n\t\t\t<attrlabl>' + headerLabel + '</attrlabl>\n' \
        + '\t\t\t<attrdef>Total variance of complex tipper component ' + headerLabel[:2] + '</attrdef>' \
        + '\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t' \
        + '<attrdomv>\n\t\t\t<rdom>\n\t\t\t<rdommin>'+ str(tipperMin[i]) + '</rdommin>\n\t\t\t<rdommax>' + str(tipperMax[i]) + '</rdommax>\n\t\t\t'\
        + '<attrunit>unitless</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>' \
        + '\n\t\t</attr>\n'
        tipAtt = tipAtt + tipAtti
        
print (tipAtt)
TipperAttributes = tipAtt

Header Label: TXR.EXP-TXR
Header Label: TXI.EXP-TXI
Header Label: TXVAR.EXP-TXV
Header Label: TYR.EXP-TYR
Header Label: TYI.EXP-TYI
Header Label: TYVAR.EXP-TYV
		<attr>
			<attrlabl>TXR.EXP</attrlabl>
			<attrdef>Real part of complex tipper component TX</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
			<rdom>
			<rdommin>-0.710182965</rdommin>
			<rdommax>-0.102175966</rdommax>
			<attrunit>unitless</attrunit>
			</rdom>
			</attrdomv>
		</attr>
		<attr>
			<attrlabl>TXI.EXP</attrlabl>
			<attrdef>Imaginary part of complex tipper component TX</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
			<rdom>
			<rdommin>-0.362285525</rdommin>
			<rdommax>0.671996057</rdommax>
			<attrunit>unitless</attrunit>
			</rdom>
			</attrdomv>
		</attr>
		<attr>
			<attrlabl>TXVAR.EXP</attrlabl>
			<attrdef>Total variance of complex tipper component TX</attrdef>
			<attrdefs>U.S. Geological Survey</attrdefs>
			<attrdomv>
			<rdom>
			<rdommin>0.00059370324<

Here we load the computed parameters
>!****COMPUTED PARAMETERS****!

In [329]:
# Import entity and attributes - !****COMPUTED PARAMETERS****! plan to break some of these individual chunks into objects/functions
# Probably will need two functions for this - one for a single list and one for the long lists with more than one column
# Get Range of COMPUTED PARAMETERS in EDI File
for k in range(list_length):
 value = list_[k] 
 if value.replace(" ", "") == '>!****COMPUTEDPARAMETERS****!':
   startIndPar = k + 1
   print ('startIndPar: ' + str(startIndPar))
 
 if value.replace(" ", "") ==  '>END':
   endIndPar = k - 1
   print ('endIndPar: ' + str(endIndPar))

#Construct Array of Channel Headers   
count = -1
currentLabel = ''
parLabel = []
parData = []
pdata = []
ampIndex = 1
noiseIndex = 1
cohIndex = 1
epIndex = 1
#Constuct a library of Headers and Values using Pandas, https://pandas.pydata.org/
parDF = pd.DataFrame(pdata)
for l in range(startIndPar,endIndPar): 
    if list_[l][0] == '>':
      ptemp = list_[l].split(" ", 1)
     #print ("ptemp:\n" + str(ptemp))
      if ptemp[0].split(">")[1] == 'SIGAMP':  
       parLabel.append((ptemp[0].split(">"))[1] + str(ampIndex))
       ampIndex = ampIndex + 1  
      elif ptemp[0].split(">")[1] == 'SIGNOISE':  
       parLabel.append((ptemp[0].split(">"))[1] + str(noiseIndex))
       noiseIndex = noiseIndex + 1 
      elif ptemp[0].split(">")[1] == 'COH':  
       parLabel.append((ptemp[0].split(">"))[1] + str(cohIndex))
       cohIndex = cohIndex + 1
      elif ptemp[0].split(">")[1] == 'EPREDCOH':  
       parLabel.append((ptemp[0].split(">"))[1] + str(epIndex))
       epIndex = epIndex + 1
      else:
        parLabel.append((ptemp[0].split(">"))[1])  
        
for l in range(startIndPar,endIndPar): 
    if list_[l][0] == '>':
     count = count + 1
     pdataTemp = list_[l+1]
     for j in range(l+2,l+9):
       pdataTemp = pdataTemp + list_[j]
       pdataTemp = re.sub(' +', ' ',pdataTemp)
       pdata = pdataTemp.split(" ")
       #print ("pdata:\n" + str(pdata))
       del pdata[0]
       del pdata[len(pdata)-1] # need to check for empty strings and delete
       pdata = np.array(pdata).astype(np.float) #convert String to floats
       pe = pd.Series(pdata)
       #print (parLabel[count]  + ':' + str(l) )
       currentLabel = parLabel[count]
    parDF[currentLabel] = pe.values
    

#pdataTemp
#parLabel
parDF
#pdata
#pe
#te.values

startIndPar: 458
endIndPar: 817


,RHOROT,RHOXX,RHOXX.ERR,RHOXY,RHOXY.ERR,RHOYX,RHOYX.ERR,RHOYY,RHOYY.ERR,PHSXX,...,SIGAMP1,SIGAMP2,SIGAMP3,SIGAMP4,SIGAMP5,SIGNOISE1,SIGNOISE2,SIGNOISE3,SIGNOISE4,SIGNOISE5
0,0.0,0.051150,0.185664,3.272341,0.205956,3.337403,0.151681,250.502670,0.155340,-159.641769,...,0.011036,0.001244,0.003698,1.664644,11.741934,1.0,1.0,0.616374,0.728700,0.762541
1,0.0,0.058304,0.138270,2.219713,0.192069,8.123118,0.082337,320.762299,0.112305,-97.765717,...,0.009055,0.001056,0.003042,0.788043,6.166758,1.0,1.0,0.603865,0.716037,0.770907
2,0.0,0.072465,0.137575,5.574779,0.134763,3.846668,0.073512,187.186493,0.090540,-82.594643,...,0.007767,0.000904,0.002579,0.512227,3.116092,1.0,1.0,0.598357,0.465856,0.781249
3,0.0,0.001084,0.434455,0.527571,0.183772,1.216701,0.087992,64.778091,0.112545,-92.165573,...,0.006099,0.000654,0.002318,0.160943,1.550458,1.0,1.0,0.548484,0.486100,0.744999
4,0.0,0.002064,0.273960,0.311891,0.215045,0.864178,0.079147,34.105083,0.121568,-112.207794,...,0.007912,0.000820,0.002828,0.139578,1.399832,1.0,1.0,0.718163,0.285745,0.751843
5,0.0,0.002855,0.205689,0.693098,0.127537,0.397530,0.110995,36.213295,0.112353,-61.598923,...,0.006313,0.000653,0.002122,0.102469,1.153880,1.0,1.0,0.434283,0.210713,0.747616
6,0.0,0.000236,1.412850,0.263701,0.409168,1.152939,0.118814,18.673809,0.285515,86.091133,...,0.003232,0.000334,0.001101,0.051736,0.507582,1.0,1.0,0.687150,0.183113,0.707375
7,0.0,0.002953,0.325413,0.228644,0.352209,1.176302,0.107814,7.763211,0.399681,-85.241684,...,0.002990,0.000314,0.001159,0.039033,0.444078,1.0,1.0,0.625888,0.160539,0.716440
8,0.0,0.000956,0.683649,0.122484,0.615757,1.311226,0.111662,10.996288,0.393150,-136.841385,...,0.003027,0.000297,0.000926,0.042919,0.501470,1.0,1.0,0.505256,0.207195,0.787456
9,0.0,0.010652,0.241199,0.746484,0.281604,1.231218,0.161795,24.064775,0.357683,-98.522423,...,0.002906,0.000297,0.001143,0.042261,0.464056,1.0,1.0,0.591679,0.151633,0.634071


In [330]:
# Now lets get the stats of the computed parameters

# Make Array of Max Values
parMax = []
for i in range (0,len(parLabel)):
 parMax.append(parDF[(parLabel[i])].max())
 print ('Computed Pararmeters Max of ' + str(parLabel[i]) + ': ' + str(parMax[i]))    

# Make Array of Min Values
parMin = []
for i in range (0,len(parLabel)):
 parMin.append(parDF[(parLabel[i])].min())
 print ('Computed Pararmeters Min: ' + str(parMin))

for i in range (0,len(parLabel)):
    print ('Header Index ' + str(parLabel[i]) + ':' + str(i))

Computed Pararmeters Max of RHOROT: 0.0
Computed Pararmeters Max of RHOXX: 0.320886433
Computed Pararmeters Max of RHOXX.ERR: 1.41284955
Computed Pararmeters Max of RHOXY: 19.2948627
Computed Pararmeters Max of RHOXY.ERR: 0.615756989
Computed Pararmeters Max of RHOYX: 35.5703697
Computed Pararmeters Max of RHOYX.ERR: 0.180633545
Computed Pararmeters Max of RHOYY: 320.762299
Computed Pararmeters Max of RHOYY.ERR: 1.30166137
Computed Pararmeters Max of PHSXX: 105.521454
Computed Pararmeters Max of PHSXX.ERR: 93.2003708
Computed Pararmeters Max of PHSXY: 154.136368
Computed Pararmeters Max of PHSXY.ERR: 40.6191788
Computed Pararmeters Max of PHSYX: 141.277328
Computed Pararmeters Max of PHSYX.ERR: 11.9157162
Computed Pararmeters Max of PHSYY: 177.738281
Computed Pararmeters Max of PHSYY.ERR: 85.8657074
Computed Pararmeters Max of TIPMAG: 4.89754725
Computed Pararmeters Max of TIPMAG.ERR: 1.00000003e+32
Computed Pararmeters Max of TIPPHS: 167.832535
Computed Pararmeters Max of TIPPHS.ERR: 

In [331]:
# assign variables for xml file find and replace
RHOROTmax = parMax[0]
RHOROTmin = parMin[0]
print ('RHOROTmax = ' + str(RHOROTmax))
print ('RHOROTmin = ' + str(RHOROTmin))
RHOXXmax = parMax[1]
RHOXXmin = parMin[1]
print ('RHOXXmax = ' + str(RHOXXmax))
print ('RHOXXmin = ' + str(RHOXXmin))
PHSXXmax = parMax[9]
PHSXXmin = parMin[9]
RHOXXERRmax = parMax[2]
RHOXXERRmin = parMin[2]
PHSXXERRmax = parMax[10]
PHSXXERRmin = parMin[10]
RHOXYmax = parMax[3]
RHOXYmin = parMin[3]
PHSXYmax = parMax[11]
PHSXYmin = parMin[11]
RHOXYERRmax = parMax[4]
RHOXYERRmin = parMin[4]
PHSXYERRmax = parMax[12]
PHSXYERRmin = parMin[12]
RHOYXmax = parMax[5]
RHOYXmin = parMin[5]
PHSYXmax = parMax[13]
PHSYXmin = parMin[13]
RHOYXERRmax = parMax[6]
RHOYXERRmin = parMin[6]
PHSYXERRmax = parMax[14]
PHSYXERRmin = parMin[14]
RHOYYmax = parMax[7]
RHOYYmin = parMin[7]
PHSYYmax = parMax[15]
PHSYYmin = parMin[15]
RHOYYERRmax = parMax[8]
RHOYYERRmin = parMin[8]
PHSYYERRmax = parMax[16]
PHSYYERRmin = parMin[16]
TIPMAGmax = parMax[17]
TIPMAGmin = parMin[17]
print ('TIPMAGmin = ' + str(TIPMAGmin))
TIPPHSmax = parMax[19]
TIPPHSmin = parMin[19]
TIPMAGERRmax = parMax[18]
TIPMAGERRmin = parMin[18]
TIPPHSERRmax = parMax[20]
TIPPHSERRmin = parMin[20]
ZSTRIKEmax = parMax[21]
ZSTRIKEmin = parMin[21]
ZSKEWmax = parMax[22]
ZSKEWmin = parMin[22]
TSTRIKEmax = parMax[23]
TSTRIKEmin = parMin[23]
COH1max = parMax[24]
COH1min = parMin[24]
COH2max = parMax[25]
COH2min = parMin[25]
COH3max = parMax[26]
COH3min = parMin[26]
COH4max = parMax[27]
COH4min = parMin[27]
EPREDCOH1max = parMax[28]
EPREDCOH1min = parMin[28]
EPREDCOH2max = parMax[29]
EPREDCOH2min = parMin[29]
SIGAMP1max = parMax[30]
SIGAMP1min = parMin[30]
SIGAMP2max = parMax[31]
SIGAMP2min = parMin[31]
SIGAMP3max = parMax[32]
SIGAMP3min = parMin[32]
SIGAMP4max = parMax[33]
SIGAMP4min = parMin[33]
SIGAMP5max = parMax[34]
SIGAMP5min = parMin[34]
SIGNOISE1max = parMax[35]
SIGNOISE1min = parMin[35]
SIGNOISE2max = parMax[36]
SIGNOISE2min = parMin[36]
SIGNOISE3max = parMax[37]
SIGNOISE3min = parMin[37]
SIGNOISE4max = parMax[38]
SIGNOISE4min = parMin[38]
SIGNOISE5max = parMax[39]
SIGNOISE5min = parMin[39]

RHOROTmax = 0.0
RHOROTmin = 0.0
RHOXXmax = 0.320886433
RHOXXmin = 0.00023646903
TIPMAGmin = 0.485399157


## Now lets get the range of values from the RSP values

## Now the raw Binary File Listing - this can be T files or W files
We will need to figure out the best way of filtering on thise - may need to build array and then delete AVG, dmp and edi file.

These are listed in the edi file as well but they are not all there.  

    ProcessingTimeSeriesUsed:
         wp01A1.bp1                                                                     
         wp01A2.bp1                                                                     
         wp01A1.sd6                                                                     
         wp01A2.sd6                                                                     
         wp01A1.sd7                                                                     
         wp01A2.sd8                                                                     
         wp01A2_3.sd9 

- Which files need to be included in the data release?
- What is the best way to get this listing?

In [332]:
#First Get the list of RSP files
rspList = glob.glob(os.path.join(mtStationPath, '*.RSP'),  recursive=True)
#rspList
ediFile = str(ediPathArray[len(ediPathArray)-1])
fileListing = ''
fileListing = fileListing + '\t\t\t\t\t\t' + ediFile + '\n' #start the file listing with the main EDI file
rspFileListing = []
for i in range(len(rspList)):
    splitRspList = rspList[i].split('\\')
    fileListing = fileListing + '\t\t\t\t\t\t' + splitRspList[len(splitRspList) - 1] + '\n'
    rspFileListing.append(splitRspList[len(splitRspList) - 1])
print (fileListing)


						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						BF6-9626.RSP
						BF6-9628.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						EF-9608X.RSP
						EF-9608Y.RSP



In [333]:
#Now finally add the processed ASCII text files to the list
txtList = glob.glob(os.path.join(mtStationPath, '*.txt'),  recursive=True)
txtListFormatted = []
#txtList
print ('Text File Only Listing:\n')
for i in range(len(txtList)):
  splitTxtList = txtList[i].split('\\')
  txtListFormatted.append(splitTxtList[len(splitTxtList) - 1])
  #print (txtListFormatted[i])  
  fileListing = fileListing + '\t\t\t\t\t\t' + splitTxtList[len(splitTxtList) - 1] + '\n'

#print ('File Listing:\n' + fileListing)
#Remove readme file from list.  May need to search an remove if it does not come up first in alphabetical order?'
del txtListFormatted[0]
print (txtListFormatted)

Text File Only Listing:

['USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_01.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_02.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_03.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_04.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_01.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_02.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_01.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_02.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC8_01.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC8_02.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC9_01.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC9_02.txt', 'USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-SD6_01.txt', 'USA-Arizona-Patagonia_Mtns-Sunnysi

In [334]:
# Create the Entity and Attributes for the .txt? listing (Note: for each station, replace sciencebase link with correct link)
txtEandA = ''
#As before, we read in the list of files

for i in range(len(txtListFormatted)):
    if txtListFormatted[i].find('-BP') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + ' </enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for raw cross power files.</eaover> '\
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for raw cross power files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for cross power content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
#            + '\n\t\t\t\t\t<udom>Header description and settings for cross power content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n' 
    if txtListFormatted[i].find('-FC') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File. ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary Fourier coefficient files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for raw Fourier coefficient files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for Fourier coefficient binary content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
#            + '\n\t\t\t\t\t<udom>Header description and settings for Fourier coefficient binary content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    if txtListFormatted[i].find('-SD') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for synchronous detection cross power files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for synchronous detection cross power files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for cross power content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t\t<attrdomv>'\
#            + '\n\t\t\t\t\t<udom>Header description and settings for cross power content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
            
    if txtListFormatted[i].find('-TS') != -1:
#        txtEandA = txtEandA + '\t\t<detailed>\n\t\t\t<enttyp>\n\t\t\t\t<enttypl>Text File ' + txtListFormatted[i] + '</enttypl>\n' \
        txtEandA = txtEandA + '\t\t<overview>\n\t\t\t<eaover>Text File, ' + txtListFormatted[i] + ' is a header file in ASCII text format for binary time series files.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#            + '\t\t\t\t<enttypd>Header file in ASCII text format for binary time series files</enttypd>' \
#            + '\n\t\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t\t</enttyp>' \
#            + '\n\t\t\t<attr>\n\t\t\t\t<attrlabl>Header Information</attrlabl>\n\t\t\t\t<attrdef>Header description and settings for time series binary content</attrdef>\n\t\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
#            + '\t\t\t\t<attrdomv>\n\t\t\t\t\t<udom>Header description and settings for time series binary content</udom>\n\t\t\t\t</attrdomv>' \
#            + '\n\t\t\t</attr>\n\t\t</detailed>\n'
    
#Add the final .txt listing for the readme file.  
#Be sure to comment this out if there is no readme.txt file included with the data release

#readmeEandA = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File readme.txt</enttypl>'\
readmeEandA = '\t\t<overview>\n\t\t<eaover>Text File, readme.txt, is a Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</eaover>'\
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\n\t\t\t<enttypd>Read Me file describing the naming format of the EDI files and that they may need to be renamed to be imported into certain software packages.</enttypd>'\
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>'\
#        + '\n\t\t</enttyp>\n\t\t</detailed>\n'
    
txtEandA = txtEandA + readmeEandA
print (txtEandA)
ALL_EandA = '' #'<eainfo>\n'   
ALL_EandA = ALL_EandA + txtEandA

		<overview>
			<eaover>Text File, USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_01.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
			<eaover>Text File, USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_02.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
			<eaover>Text File, USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_03.txt is a header file in ASCII text format for raw cross power files.</eaover> 
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
			<eaover>Text File, USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_04.txt is a header file in ASCII text form

# Now get values and stats on the .RSP files that are listed to add to the Ent. and Att. information

### These files are all fixed width format (666) but it looks like the BFS*.RSP and EF*.RSP are slightly different beasts that have different formats

In [335]:
# Load the BFS*.RSP files into pandasand create entities and attributes
allRspEandP = ''
strFreqMaxBSP = ''
strAmpMaxBSP = ''
strGammaMaxBSP = ''
strFreqMinBSP = ''
strAmpMinBSP = ''
strGammaMinBSP = ''

strFreqMaxESP = ''
strFreqMinESP = ''
strAmp1MaxESP = ''
strAmp1MinESP = ''
strAmp2MaxESP = ''
strAmp2MinESP = ''
strAmp3MaxESP = ''
strAmp3MinESP = ''
strAmp4MaxESP = ''
strAmp4MinESP = ''
strPhz1MaxESP = ''
strPhz1MinESP = ''
strPhz2MaxESP = ''
strPhz2MinESP = ''
strPhz3MaxESP = ''
strPhz3MinESP = ''
strPhz4MaxESP = ''
strPhz4MinESP = ''

intNumberBSF = 0
intNumberEFF = 0
for i in range(len(rspList)):
#for i in range(3):
    if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        bfRSP = pd.read_fwf(rspList[i], widths=[6,6,6], skiprows=5, parse_dates=True).rename(columns={'31':'Freq', '1':'Amp', 'Unnamed: 2':'Gamma'})
#print (rspList[1])
        #print (bfRSP)    
    
# Now lets get the stats of the bfRSP data

# Make Array of Min and Max Values
        FreqMaxRSP = bfRSP['Freq'].max()
        strFreqMaxBSP = str(FreqMaxRSP)
        AmpMaxRSP = bfRSP['Amp'].max()
        strAmpMaxBSP = str(AmpMaxRSP)
        GammaMaxRSP = bfRSP['Gamma'].max()
        strGammaMaxBSP = str(GammaMaxRSP)
        FreqMinRSP = bfRSP['Freq'].min()
        strFreqMinBSP = str(FreqMinRSP)
        AmpMinRSP = bfRSP['Amp'].min()
        strAmpMinBSP = str(AmpMinRSP)
        GammaMinRSP = bfRSP['Gamma'].min()
        strGammaMinBSP = str(GammaMinRSP)
# now print RSP entity and attribute    
        bspEandA = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + rspFileListing [i] + '</enttypl>\n' \
        + '\t\t\t<enttypd>System Calibration File</enttypd>\n\t\t\t<enttypds>Electromagnetic Instruments (EMI)</enttypds>\n\t\t</enttyp>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Freq</attrlabl>\n\t\t\t<attrdef>Frequency - Hz</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strFreqMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strFreqMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Hz</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Amp</attrlabl>\n\t\t\t<attrdef>Amplitude - Volts/Gamma</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strAmpMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strAmpMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Volts/Gamma</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Phz</attrlabl>\n\t\t\t<attrdef>Phase - Degrees</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strGammaMinBSP + '</rdommin>\n\t\t\t\t<rdommax>' + strGammaMaxBSP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Degrees</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t</detailed>'
        #print ('i Loop = ' + str(i))
        allRspEandP = allRspEandP +  bspEandA + '\n'
    if rspList[i].find('EF') > 0:
        #print ('i Loop = ' + str(i) + ' EF File Found')
        efRSP = pd.read_fwf(rspList[i], widths=[6,6,6,6,6,6,6,6,6], skiprows=5, parse_dates=True)\
        .rename(columns={'42':'Freq', '4':'Amp1', 'Low F':'Phz1', 'requen':'Amp2', 'cy':'Phz2', '10Hz':'Amp3', 'Out':'Phz3',\
        'Unnamed: 7':'Amp4', 'Unnamed: 8':'Phz4'})
#Delete headers from data to remove all strings from Pandas Data Frame

        efRSP = efRSP[efRSP.Amp2 != 'Freque']
        efRSP = efRSP[efRSP.Phz1 != 'Low F']
        #print (str(efRSP))   
# Make Array of Min and Max Values
        strFreqMaxESP = str(efRSP['Freq'].max())
        strFreqMinESP = str(efRSP['Freq'].min())
        strAmp1MaxESP = str(efRSP['Amp1'].max())
        strAmp1MinESP = str(efRSP['Amp1'].min())
        strAmp2MaxESP = str(efRSP['Amp2'].max())
        strAmp2MinESP = str(efRSP['Amp2'].min())
        strAmp3MaxESP = str(efRSP['Amp3'].max())
        strAmp3MinESP = str(efRSP['Amp3'].min())
        strAmp4MaxESP = str(efRSP['Amp4'].max())
        strAmp4MinESP = str(efRSP['Amp4'].min())
        strPhz1MaxESP = str(efRSP['Phz1'].max())
        print ('strPhz1MaxESP = ' + (strPhz1MaxESP))
        strPhz1MinESP = str(efRSP['Phz1'].min())
        strPhz2MaxESP = str(efRSP['Phz2'].max())
        strPhz2MinESP = str(efRSP['Phz2'].min())
        strPhz3MaxESP = str(efRSP['Phz3'].max())
        print ('strPhz3MaxESP = ' + strPhz3MaxESP)
        strPhz3MinESP = str(efRSP['Phz3'].min())
        strPhz4MaxESP = str(efRSP['Phz4'].max())
        print ('strPhz4MaxESP = ' + strPhz4MaxESP)
        strPhz4MinESP = str(efRSP['Phz4'].min())
        espEandA = '\t<detailed>\n\t\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + rspFileListing [i] + '</enttypl>\n' \
        + '\t\t\t<enttypd>System Calibration File</enttypd>\n\t\t\t<enttypds>Electromagnetic Instruments (EMI)</enttypds>\n\t\t</enttyp>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Freq</attrlabl>\n\t\t\t<attrdef>Frequency - Hz</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strFreqMinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strFreqMaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Hz</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t<attr>\n\t\t\t<attrlabl>Amp</attrlabl>\n\t\t\t<attrdef>Amplitude - Volts</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strAmp1MinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strAmp1MaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Volts</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t\t\t<attr>\n\t\t\t<attrlabl>Phz</attrlabl>\n\t\t\t<attrdef>Phase - Degrees</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>' \
        + '\n\t\t\t<attrdomv>\n\t\t\t\t<rdom>\n\t\t\t\t<rdommin>' \
        + strPhz1MinESP + '</rdommin>\n\t\t\t\t<rdommax>' + strPhz1MaxESP + '</rdommax>\n\t\t\t\t' \
        + '<attrunit>Degrees</attrunit>\n\t\t\t</rdom>\n\t\t\t</attrdomv>\n\t\t</attr>\n' \
        + '\t</detailed>'
        #print ('i Loop = ' + str(i))
        allRspEandP = allRspEandP +  espEandA + '\n' 
        #print (str(allRspEandP)) 

     
EandA = allRspEandP # [-:2] # this removes the last characters of the string to get rid of the last line return.  
ALL_EandA = ALL_EandA +  allRspEandP


strPhz1MaxESP = 9.1
strPhz3MaxESP = 9.2
strPhz4MaxESP = 82.9
strPhz1MaxESP = 9.1
strPhz3MaxESP = 9.1
strPhz4MaxESP = 82.8
strPhz1MaxESP = 9.0
strPhz3MaxESP = 82.6
strPhz4MaxESP = 82.7
strPhz1MaxESP = 9.1
strPhz3MaxESP = 9.1
strPhz4MaxESP = 82.8


## Now let's work on the SD mode files

### Overview of SD files,

The prefix file naming convention for the SD mode AMT non-transmitter files are:
AANNNRN where AA is the survey area, NNN is the site number, R is the run "number" (run A, run B, run C, etc.), and N is a subset run number (first run is A1, second run is A2, third run is A3, etc.)

The suffix file naming convention for the SD mode AMT non-transmitter files are:
SD6 sample frequencies are always 79, 90, 100, 150, 210, 270, 340, 460, 580 Hertz
SD7 sample frequencies are always 340, 460, 580, 720, 885, 1170 Hertz
SD8 sample frequencies are always 1170, 1500, 1870, 2200, 2730, 3550, 4900, 6500, 9000 Hertz
SD9 sample frequencies are always 6500, 9000, 11590, 15290, 19500, 23370 Hertz

The prefix file naming convention for the SD mode AMT transmitter files are:
AANNNRN where AA is the survey area, NNN is the site number, R is the run "number" (run A, run B, run C, etc.), and N is a subset run "number" (first run is AA, second run is AB, third run is AC, etc.)

The suffix file naming convention for the SD mode AMT transmitter files are:
SDA sample frequency is always 960 Hertz
SDB sample frequency is always 1200 Hertz
SDC sample frequency is always 1870 Hertz
SDD sample frequency is always 2420 Hertz
SDE sample frequency is always 2730 Hertz
SDF sample frequency is always 3550 Hertz
SDG sample frequency is always 5210 Hertz
SDH sample frequency is always 6850 Hertz
SDI sample frequency is always 11590 Hertz
SDJ sample frequency is always 15920 Hertz
SDK sample frequency is always 23370 Hertz

The file format of the SD mode files are described in each of the text files (last block of information within each file). It is the same data format regardless of whether or not a transmitter was used to collect the data. We must have been looking at the text file for the FC (Fourier Coefficient) files in your office, so I was confused that I hadn't described the data format (it is there in the text files for all SD files).



In [336]:
#First get a listing of the SD mode files
#Now finally add the processed ASCII text files to the list
sdList = glob.glob(os.path.join(mtStationPath, '*.sd?'),  recursive=True)
#sdaList
sdFileList = ''
for i in range(len(sdList)):
  splitSdList = sdList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitSdList[len(splitSdList) - 1] + '\n'
  sdFileList = sdFileList + splitSdList[len(splitSdList) - 1] + '\n'

#print ('File ListingfileListing:\n' + fileListing)
print ('SD file listing:\n' + sdFileList)


SD file listing:
SP07A1.SD6
SP07A1.SD7
SP07A1.SD8
SP07A1.SD9
SP07A2.SD6
SP07A2.SD7
SP07A2.SD8
SP07A2.SD9



In [337]:
#As before, we read in the list of files
# Need to delete files in array with no data
##del sdList[0]
##del sdList[1]
##del sdList[2]
for i in range(len(sdList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfSD = pd.read_fwf(sdList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'79.0000  32.':'Amp1', '00000   20   1':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        print (dfSD)        
    

               Amp1             Amp2       Amp3      Amp4       Amp5
0   20.45476446e-01              0.0   3.374460 -0.341226   5.631117
1               0.0  41.16607691e-03  -0.033248  0.073619  -0.048032
2   13.76590570e-04              0.0  -0.023132  0.010954  -0.039989
3   14.22053317e-03  -6.45069708e-04  -0.000155  0.000321   0.000000
4   13.35064448e-03  -4.82450593e-02   0.030064 -0.077438   0.001056
5   -7.58170520e-04  -4.10461200e-04   0.000475  0.001231   0.000000
6      90.0000  32.   00000   20   1        NaN       NaN        NaN
7   52.40583058e-02              0.0   0.863144 -0.085644   1.437999
8               0.0  10.19085622e-03  -0.008392  0.018190  -0.012165
9   33.46586119e-05              0.0  -0.005774  0.002836  -0.009970
10  37.27944633e-04  -1.58350147e-04  -0.000037  0.000079   0.000000
11  32.60482858e-04  -1.16816306e-02   0.007299 -0.018708   0.000253
12  -1.75937797e-04  -9.99012793e-05   0.000112  0.000285   0.000000
13    100.0000  32.   00000   20  

In [338]:
#listing of the Fourier Coefficient files (FC) files.
#These are Binary so we don't need to come up with the range of the values.
fcList = glob.glob(os.path.join(mtStationPath, '*.fc?'),  recursive=True)
#fcList
for i in range(len(fcList)):
  splitFcList = fcList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitFcList[len(splitFcList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						BF6-9626.RSP
						BF6-9628.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						EF-9608X.RSP
						EF-9608Y.RSP
						readme.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_03.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_04.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC8_01.txt
						USA-Arizona-Patagonia_Mtns-Sunny

In [339]:
# Create the Entity and Attributes for the FC listing
fcEandAi = ''
fcEandA = ''
for i in range(len(fcList)):
    splitFcList = fcList[i].split('\\')
#    fcEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Binary File ' + splitFcList[len(splitFcList) - 1] + '</enttypl>\n' \
    fcEandAi = '\t\t<overview>\n\t\t<eaover>Binary File, ' + splitFcList[len(splitFcList) - 1] + ' is a raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Binary Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
#        + '\t\t\t<attrdomv>\n\t\t\t\t<udom>Binary Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
    fcEandA = fcEandA + fcEandAi

ALL_EandA = ALL_EandA + fcEandA    
print (fcEandA)        
        
        
        
        

		<overview>
		<eaover>Binary File, SP07A1.FC6 is a raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SP07A1.FC7 is a raw Fourier Coefficient Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SP07A1.FC8 is a raw Fourier Coefficient Binary Data File. The header

In [340]:
#listing of the Time Series TS? files
#Now finally add the processed ASCII text files to the list
tsList = glob.glob(os.path.join(mtStationPath, '*.ts?'),  recursive=True)
#tsList
for i in range(len(tsList)):
  splitTsList = tsList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitTsList[len(splitTsList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						BF6-9626.RSP
						BF6-9628.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						EF-9608X.RSP
						EF-9608Y.RSP
						readme.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_03.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_04.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC8_01.txt
						USA-Arizona-Patagonia_Mtns-Sunny

In [341]:
# Create the Entity and Attributes for the Time series .TS? listing

tsEandAi = ''
tsEandA = ''
for i in range(len(tsList)):
    splitTsList = tsList[i].split('\\')
#    tsEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Binary File ' + splitTsList[len(splitTsList) - 1] + '</enttypl>\n' \
    tsEandAi = '\t\t<overview>\n\t\t<eaover>Binary File, ' + splitTsList[len(splitTsList) - 1] + ' is a raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>' \
#        + '\n\t\t</enttyp>\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Binary Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n'\
#        + '\t\t\t<attrdomv>\n\t\t\t\t<udom>Binary Data Value</udom>\n\t\t\t\t</attrdomv>' \
#        + '\t\t</attr>\n\t</detailed>\n'
    tsEandA = tsEandA + tsEandAi

ALL_EandA = ALL_EandA + tsEandA
print (tsEandA)

		<overview>
		<eaover>Binary File, SP07A1.TS1 is a raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SP07A2.TS1 is a raw Time Series Binary Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Binary File, SP07A3.TS1 is a raw Time Series Binary Data File. The header information for this fi

In [342]:
#listing of the BP? files
#Now finally add the processed ASCII text files to the list
bpList = glob.glob(os.path.join(mtStationPath, '*.bp?'),  recursive=True)
#bpList
for i in range(len(bpList)):
  splitBpList = bpList[i].split('\\')
  fileListing = fileListing + '\t\t\t\t\t\t' + splitBpList[len(splitBpList) - 1] + '\n'

print ('File ListingfileListing:\n' + fileListing)

File ListingfileListing:
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.edi
						BF6-9621.RSP
						BF6-9624.RSP
						BF6-9625.RSP
						BF6-9626.RSP
						BF6-9628.RSP
						EF-9515X.RSP
						EF-9515Y.RSP
						EF-9608X.RSP
						EF-9608Y.RSP
						readme.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_03.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-BP_High_04.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC6_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_01.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC7_02.txt
						USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07-FC8_01.txt
						USA-Arizona-Patagonia_Mtns-Sunny

In [343]:
# Create the Entity and Attributes for the .BP? listing

#As before, we read in the list of files

print (bpList)


for i in range(len(bpList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfBP = pd.read_fwf(bpList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        #print (dfBP)        

bpEandAi = ''
bpEandA = ''
for i in range(len(bpList)):
    splitBpList = bpList[i].split('\\')
#    bpEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitBpList[len(splitBpList) - 1] + '</enttypl>\n' \
    bpEandAi = '\t\t<overview>\n\t\t<eaover>Text File, ' + splitBpList[len(splitBpList) - 1] + ' is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
#        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
    bpEandA = bpEandA + bpEandAi
    
ALL_EandA = ALL_EandA +  bpEandA
print (bpEandA)

['C:\\CurrentWork\\DataReleases\\patagoniaAZ\\Data\\AMT07\\SP07A1.BP1', 'C:\\CurrentWork\\DataReleases\\patagoniaAZ\\Data\\AMT07\\SP07A2.BP1', 'C:\\CurrentWork\\DataReleases\\patagoniaAZ\\Data\\AMT07\\SP07A3.BP1', 'C:\\CurrentWork\\DataReleases\\patagoniaAZ\\Data\\AMT07\\SP07A4.BP1']
		<overview>
		<eaover>Text File, SP07A1.BP1 is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Text File, SP07A2.BP1 is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the elec

In [344]:
#listing of the SD? files
#Now finally add the processed ASCII text files to the list
sdList = glob.glob(os.path.join(mtStationPath, '*.sd?'),  recursive=True)
#sdList
for i in range(len(sdList)):
  splitSdList = sdList[i].split('\\')
  #fileListing = fileListing + '\t\t\t\t\t\t' + splitSdList[len(splitSdList) - 1] + '\n'
  print ('SD? File:\n' + splitSdList[len(splitSdList) - 1])


SD? File:
SP07A1.SD6
SD? File:
SP07A1.SD7
SD? File:
SP07A1.SD8
SD? File:
SP07A1.SD9
SD? File:
SP07A2.SD6
SD? File:
SP07A2.SD7
SD? File:
SP07A2.SD8
SD? File:
SP07A2.SD9


In [345]:
#SD files Entity and Attributes
for i in range(len(sdList)):
#for i in range(3):
    #if rspList[i].find('BF') > 0: #Do this if the file has a BF style Fixed Width Format
        dfBP = pd.read_fwf(sdList[i], widths=[15,15,15,15,15], skiprows=27, parse_dates=True).rename(columns={'4.3945   1.':'Amp1', '85697    1   2':'Amp2', 'Unnamed: 2':'Amp3', 'Unnamed: 3':'Amp4', 'Unnamed: 4':'Amp5'})
#Now strip out the header rows in the data as to not screw up calculating the range
        #dfSD = dfSD[dfSD.Amp4 != '00000   20   1']
        #print (dfBP)        

sdEandA = ''
sdEandAi = ''

for i in range(len(sdList)):
    splitSdList = sdList[i].split('\\')
#    sdEandAi = '\t\t<detailed>\n\t\t<enttyp>\n\t\t\t<enttypl>Text File ' + splitSdList[len(splitSdList) - 1] + '</enttypl>\n' \
    sdEandAi = '\t\t<overview>\n\t\t<eaover>Text File, ' + splitSdList[len(splitSdList) - 1] + ' is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>' \
            + '\n\t\t\t<eadetcit>' + childURL + '</eadetcit>\n\t\t</overview>\n' \
#        + '\t\t\t<enttypd>Raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</enttypd>' \
#        + '\n\t\t\t<enttypds>U.S. Geological Survey</enttypds>\n\t\t</enttyp>' \
#        + '\n\t\t<attr>\n\t\t\t<attrlabl>Data Value</attrlabl>\n\t\t\t<attrdef>Data Value</attrdef>\n\t\t\t<attrdefs>U.S. Geological Survey</attrdefs>\n\t\t\t<attrdomv>'\
#        + '\n\t\t\t\t<udom>Data Value</udom>\n\t\t\t</attrdomv>' \
#        + '\n\t\t</attr>\n\t\t</detailed>\n'
    sdEandA = sdEandA + sdEandAi
    
    
ALL_EandA = ALL_EandA +  sdEandA
#print (len(sdList))
#print ((sdList))
print (sdEandA)

		<overview>
		<eaover>Text File, SP07A1.SD6 is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Text File, SP07A1.SD7 is a raw Cross Power Data File. The header information for this file contains field notes including survey identifier, personnel, date/time, weather, indication of a remote MT site, additional comments and sensor configuration for the electric and magnetic field channels.</eaover>
			<eadetcit>https://www.sciencebase.gov/catalog/item/5e715428e4b01d5092688a8e</eadetcit>
		</overview>
		<overview>
		<eaover>Text File, SP07A1.SD8 is a raw Cross Power Data File. The header information for this file contains field notes inc

# Populate Metadata Template

In [346]:
#Load XML Metadata Template File and Read It
metaData = os.path.join(mtMetaDataTemplatePath, mtMetaDataTemplateName)
print ('Metadata path: ' + mtMetaDataTemplatePath + '\n')
xmlTemplateFile = open(metaData, 'r')
metaDataContent = xmlTemplateFile.readlines()
print(metaDataContent)
xmlTemplateFile.close()


Metadata path: C:\CurrentWork\DataReleases\patagoniaAZ

['<?xml version="1.0" encoding="UTF-8"?>\n', '<metadata>\n', '\t<idinfo>\n', '\t\t<citation>\n', '\t\t\t<citeinfo>\n', '\t\t\t\t<origin>Reitman, J.</origin>\n', '\t\t\t\t<origin>Rodriguez, B.D.</origin>\n', '\t\t\t\t<origin>Brown, P.J.</origin>\t\t\t\t\t\t\t\t\n', '\t\t\t\t<pubdate>2020</pubdate>\n', '\t\t\t\t<title>{title}</title>\n', '\t\t\t\t<edition>1</edition>\n', '\t\t\t\t<geoform>ASCII and Binary Digital Data</geoform>\n', '\t\t\t\t<pubinfo>\n', '\t\t\t\t\t<pubplace>Denver, CO</pubplace>\n', '\t\t\t\t\t<publish>U.S. Geological Survey</publish>\n', '\t\t\t\t</pubinfo>\n', '\t\t\t\t<othercit>Additional information about Originators: Reitman, J., http://orcid.org/0000-0003-3551-9884; Rodriguez, B.D., http://orcid.org/0000-0002-2263-611X; Brown, P.J., http://orcid.org/0000-0002-2415-7462</othercit>\n', '\t\t\t\t<onlink>https://doi.org/10.5066/P91599D6</onlink>\n', '\t\t\t</citeinfo>\n', '\t\t</citation>\n', '\t\t<descript>\n', 

In [347]:
# Replace values of current metadata template with the appropriate values.  
# All of this input should have been defined when going through the steps outlined above.
lineString = ''
newMetaDataContent = metaDataContent
splitFileName = ediList[0].split('.')
myfilename = splitFileName[0] + '.xml'
xmlFile = open(myfilename,"w+")
print(myfilename)
#print(keywords.value)
for i in range(len(metaDataContent)):
    lineString = metaDataContent[i]
    if lineString.find('{title}'):
     lineString = lineString.replace('{title}', citTit + '; Station ' + strChildID)
    
    if lineString.find('{abstract}'):
     lineString = lineString.replace('{abstract}', purposeClean)
    
    if lineString.find('{purpose}'):
     lineString = lineString.replace('{purpose}', descriptionClean)
    
    if lineString.find('{BeginFileListingHere}'):
     lineString = lineString.replace('{BeginFileListingHere}', fileListing)
    
    if lineString.find('{keywords}'):
     lineString = lineString.replace('{keywords}', keywords.value)
    
    if lineString.find('{CollectionDate}'):
     lineString = lineString.replace('{CollectionDate}', strCollectionDate)
    
    if lineString.find('{CombinationDate}'):
     lineString = lineString.replace('{CombinationDate}', strCombinationDate)

    if lineString.find('{ConversionDate}'):
     lineString = lineString.replace('{ConversionDate}', strConversionDate)

    if lineString.find('{RotationDate}'):
     lineString = lineString.replace('{RotationDate}', strRotationDate)
    
    if lineString.find('{HarvestDate}'):
     lineString = lineString.replace('{HarvestDate}', strHarvestDate)
   
    if lineString.find('{begdate}'):
     lineString = lineString.replace('{begdate}', begdate)
    
    if lineString.find('{enddate}'):
     lineString = lineString.replace('{enddate}', enddate)
    
    if lineString.find('{SiteLon}'):
     lineString = lineString.replace('{SiteLon}', sitLon)
    
    if lineString.find('{SiteLon}'):
     lineString = lineString.replace('{SiteLat}', sitLat)
    
    if lineString.find('{EandA}'):
     lineString = lineString.replace('{EandA}', ALL_EandA)
    
    if lineString.find('{frequencyMin}'):
     lineString = lineString.replace('{frequencyMin}', str(frequencyMin))
    
    if lineString.find('{frequencyMax}'):
     lineString = lineString.replace('{frequencyMax}', str(frequencyMax))
    
    if lineString.find('{rotationMax}'):
     lineString = lineString.replace('{rotationMax}', str(rotationMax))
    
    if lineString.find('{rotationMin}'):
     lineString = lineString.replace('{rotationMin}', str(rotationMin)) 
    
    if lineString.find('{EdiFileName}'):
     lineString = lineString.replace('{EdiFileName}', EdiFileName)
    
    if lineString.find('{RHOROTmax}'):
     lineString = lineString.replace('{RHOROTmax}', str(RHOROTmax))
    
    if lineString.find('{RHOROTmin}'):
     lineString = lineString.replace('{RHOROTmin}', str(RHOROTmin))
    
    if lineString.find('{RHOXXmax}'):
     lineString = lineString.replace('{RHOXXmax}', str(RHOXXmax))
    
    if lineString.find('{RHOXXmin}'):
     lineString = lineString.replace('{RHOXXmin}', str(RHOXXmin)) 
    
    if lineString.find('{PHSXXmax}'):
     lineString = lineString.replace('{PHSXXmax}', str(PHSXXmax))
    
    if lineString.find('{PHSXXmin}'):
     lineString = lineString.replace('{PHSXXmin}', str(PHSXXmin))
    
    if lineString.find('{RHOXXERRmax}'):
     lineString = lineString.replace('{RHOXXERRmax}', str(RHOXXERRmax))
    
    if lineString.find('{RHOXXERRmin}'):
     lineString = lineString.replace('{RHOXXERRmin}', str(RHOXXERRmin))
    
    if lineString.find('{PHSXXERRmax}'):
     lineString = lineString.replace('{PHSXXERRmax}', str(PHSXXERRmax))
    
    if lineString.find('{PHSXXERRmin}'):
     lineString = lineString.replace('{PHSXXERRmin}', str(PHSXXERRmin))
    
    if lineString.find('{RHOXYmax}'):
     lineString = lineString.replace('{RHOXYmax}', str(RHOXYmax))
    
    if lineString.find('{RHOXYmin}'):
     lineString = lineString.replace('{RHOXYmin}', str(RHOXYmin))
    
    if lineString.find('{PHSXYmax}'):
     lineString = lineString.replace('{PHSXYmax}', str(PHSXYmax))
    
    if lineString.find('{PHSXYmin}'):
     lineString = lineString.replace('{PHSXYmin}', str(PHSXYmin))
    
    if lineString.find('{RHOXYERRmax}'):
     lineString = lineString.replace('{RHOXYERRmax}', str(RHOXYERRmax))
    
    if lineString.find('{RHOXYERRmin}'):
     lineString = lineString.replace('{RHOXYERRmin}', str(RHOXYERRmin))
    
    if lineString.find('{PHSXYERRmax}'):
     lineString = lineString.replace('{PHSXYERRmax}', str(PHSXYERRmax))
    
    if lineString.find('{PHSXYERRmin}'):
     lineString = lineString.replace('{PHSXYERRmin}', str(PHSXYERRmin))
    
    if lineString.find('{RHOYXmax}'):
     lineString = lineString.replace('{RHOYXmax}', str(RHOYXmax))
    
    if lineString.find('{RHOYXmin}'):
     lineString = lineString.replace('{RHOYXmin}', str(RHOYXmin))
    
    if lineString.find('{PHSYXmax}'):
     lineString = lineString.replace('{PHSYXmax}', str(PHSYXmax))
    
    if lineString.find('{PHSYXmin}'):
     lineString = lineString.replace('{PHSYXmin}', str(PHSYXmin))
    
    if lineString.find('{RHOYXERRmax}'):
     lineString = lineString.replace('{RHOYXERRmax}', str(RHOYXERRmax))
    
    if lineString.find('{RHOYXERRmin}'):
     lineString = lineString.replace('{RHOYXERRmin}', str(RHOYXERRmin))
    
    if lineString.find('{PHSYXERRmax}'):
     lineString = lineString.replace('{PHSYXERRmax}', str(PHSYXERRmax))
    
    if lineString.find('{PHSYXERRmin}'):
     lineString = lineString.replace('{PHSYXERRmin}', str(PHSYXERRmin))
    
    if lineString.find('{RHOYYmax}'):
     lineString = lineString.replace('{RHOYYmax}', str(RHOYYmax))
    
    if lineString.find('{RHOYYmin}'):
     lineString = lineString.replace('{RHOYYmin}', str(RHOYYmin))
    
    if lineString.find('{PHSYYmax}'):
     lineString = lineString.replace('{PHSYYmax}', str(PHSYYmax))
    
    if lineString.find('{PHSYYmin}'):
     lineString = lineString.replace('{PHSYYmin}', str(PHSYYmin))
    
    if lineString.find('{RHOYYERRmax}'):
     lineString = lineString.replace('{RHOYYERRmax}', str(RHOYYERRmax))
    
    if lineString.find('{RHOYYERRmin}'):
     lineString = lineString.replace('{RHOYYERRmin}', str(RHOYYERRmin))
    
    if lineString.find('{PHSYYERRmax}'):
     lineString = lineString.replace('{PHSYYERRmax}', str(PHSYYERRmax))
    
    if lineString.find('{PHSYYERRmin}'):
     lineString = lineString.replace('{PHSYYERRmin}', str(PHSYYERRmin))
    
    if lineString.find('{TIPMAGmax}'):
     lineString = lineString.replace('{TIPMAGmax}', str(TIPMAGmax))
    
    if lineString.find('{TIPMAGmin}'):
     lineString = lineString.replace('{TIPMAGmin}', str(TIPMAGmin))
    
    if lineString.find('{TIPPHSmax}'):
     lineString = lineString.replace('{TIPPHSmax}', str(TIPPHSmax))
    
    if lineString.find('{TIPPHSmin}'):
     lineString = lineString.replace('{TIPPHSmin}', str(TIPPHSmin))
    
    if lineString.find('{TipperAttributes}'):
     lineString = lineString.replace('{TipperAttributes}', str(TipperAttributes))
    
    if lineString.find('{ImpedanceAttributes}'):
     lineString = lineString.replace('{ImpedanceAttributes}', str(ImpedanceAttributes))
    
    if lineString.find('{TIPMAGERRmax}'):
     lineString = lineString.replace('{TIPMAGERRmax}', str(TIPMAGERRmax))
    
    if lineString.find('{TIPMAGERRmin}'):
     lineString = lineString.replace('{TIPMAGERRmin}', str(TIPMAGERRmin))
    
    if lineString.find('{TIPPHSERRmax}'):
     lineString = lineString.replace('{TIPPHSERRmax}', str(TIPPHSERRmax))
    
    if lineString.find('{TIPPHSERRmin}'):
     lineString = lineString.replace('{TIPPHSERRmin}', str(TIPPHSERRmin))
    
    if lineString.find('{ZSTRIKEmax}'):
     lineString = lineString.replace('{ZSTRIKEmax}', str(ZSTRIKEmax))
    
    if lineString.find('{ZSTRIKEmin}'):
     lineString = lineString.replace('{ZSTRIKEmin}', str(ZSTRIKEmin))
    
    if lineString.find('{ZSKEWmax}'):
     lineString = lineString.replace('{ZSKEWmax}', str(ZSKEWmax))
    
    if lineString.find('{ZSKEWmin}'):
     lineString = lineString.replace('{ZSKEWmin}', str(ZSKEWmin))
    
    if lineString.find('{TSTRIKEmax}'):
     lineString = lineString.replace('{TSTRIKEmax}', str(TSTRIKEmax))
    
    if lineString.find('{TSTRIKEmin}'):
     lineString = lineString.replace('{TSTRIKEmin}', str(TSTRIKEmin))
    
    if lineString.find('{COH1max}'):
     lineString = lineString.replace('{COH1max}', str(COH1max))
    
    if lineString.find('{COH1min}'):
     lineString = lineString.replace('{COH1min}', str(COH1min))
    
    if lineString.find('{COH2max}'):
     lineString = lineString.replace('{COH2max}', str(COH2max))
    
    if lineString.find('{COH2min}'):
     lineString = lineString.replace('{COH2min}', str(COH2min))
    
    if lineString.find('{COH3max}'):
     lineString = lineString.replace('{COH3max}', str(COH3max))
    
    if lineString.find('{COH3min}'):
     lineString = lineString.replace('{COH3min}', str(COH3min))
    
    if lineString.find('{COH4max}'):
     lineString = lineString.replace('{COH4max}', str(COH4max))
    
    if lineString.find('{COH4min}'):
     lineString = lineString.replace('{COH4min}', str(COH4min))
    
    if lineString.find('{EPREDCOH1max}'):
     lineString = lineString.replace('{EPREDCOH1max}', str(EPREDCOH1max))
    
    if lineString.find('{EPREDCOH1min}'):
     lineString = lineString.replace('{EPREDCOH1min}', str(EPREDCOH1min))
    
    if lineString.find('{EPREDCOH2max}'):
     lineString = lineString.replace('{EPREDCOH2max}', str(EPREDCOH2max))
    
    if lineString.find('{EPREDCOH2min}'):
     lineString = lineString.replace('{EPREDCOH2min}', str(EPREDCOH2min))
    
    if lineString.find('{SIGAMP1max}'):
     lineString = lineString.replace('{SIGAMP1max}', str(SIGAMP1max))
    
    if lineString.find('{SIGAMP1min}'):
     lineString = lineString.replace('{SIGAMP1min}', str(SIGAMP1min))
    
    if lineString.find('{SIGAMP2max}'):
     lineString = lineString.replace('{SIGAMP2max}', str(SIGAMP2max))
    
    if lineString.find('{SIGAMP2min}'):
     lineString = lineString.replace('{SIGAMP2min}', str(SIGAMP2min))
    
    if lineString.find('{SIGAMP3max}'):
     lineString = lineString.replace('{SIGAMP3max}', str(SIGAMP3max))
    
    if lineString.find('{SIGAMP3min}'):
     lineString = lineString.replace('{SIGAMP3min}', str(SIGAMP3min))
    
    if lineString.find('{SIGAMP4max}'):
     lineString = lineString.replace('{SIGAMP4max}', str(SIGAMP4max))
    
    if lineString.find('{SIGAMP4min}'):
     lineString = lineString.replace('{SIGAMP4min}', str(SIGAMP4min))
    
    if lineString.find('{SIGAMP5max}'):
     lineString = lineString.replace('{SIGAMP5max}', str(SIGAMP5max))
    
    if lineString.find('{SIGAMP5min}'):
     lineString = lineString.replace('{SIGAMP5min}', str(SIGAMP5min))
    
    if lineString.find('{SIGNOISE1max}'):
     lineString = lineString.replace('{SIGNOISE1max}', str(SIGNOISE1max))
    
    if lineString.find('{SIGNOISE1min}'):
     lineString = lineString.replace('{SIGNOISE1min}', str(SIGNOISE1min))
   
    if lineString.find('{SIGNOISE2max}'):
     lineString = lineString.replace('{SIGNOISE2max}', str(SIGNOISE2max))
    
    if lineString.find('{SIGNOISE2min}'):
     lineString = lineString.replace('{SIGNOISE2min}', str(SIGNOISE2min))
   
    if lineString.find('{SIGNOISE3max}'):
     lineString = lineString.replace('{SIGNOISE3max}', str(SIGNOISE3max))
    
    if lineString.find('{SIGNOISE3min}'):
     lineString = lineString.replace('{SIGNOISE3min}', str(SIGNOISE3min))
   
    if lineString.find('{SIGNOISE4max}'):
     lineString = lineString.replace('{SIGNOISE4max}', str(SIGNOISE4max))
    
    if lineString.find('{SIGNOISE4min}'):
     lineString = lineString.replace('{SIGNOISE4min}', str(SIGNOISE4min))
   
    if lineString.find('{SIGNOISE5max}'):
     lineString = lineString.replace('{SIGNOISE5max}', str(SIGNOISE5max))
    
    if lineString.find('{SIGNOISE5min}'):
     lineString = lineString.replace('{SIGNOISE5min}', str(SIGNOISE5min))
   
    # {county}

    else:
     lineString = lineString
    xmlFile.write(lineString)
    
    #print (lineString)
     
    
    
#for r in (metaDataContent):
    #newMetaDataContent = metaDataContent.replace('{title}', drTitle)
    #newMetaDataContent = metaDataContent.replace('{keywords}', keywords.value)
xmlFile.close()

print ('Creation of new metadata file is complete\n\n') 
#Load EDI File and Read It
##checkFile = open(open(myfilename, 'r')
##checkFileContent = checkFile.read()
##checkFile.close()
##print(checkFileContent)

C:\CurrentWork\DataReleases\patagoniaAZ\Data\AMT07\USA-Arizona-Patagonia_Mtns-Sunnyside_Porphyry-2010-AMT07.xml
Creation of new metadata file is complete




### Check this file to see if it is valid against the FGDC metadata standard (FGDC-STD-001-1998)

## https://mrdata.usgs.gov/validation/

In [ ]:
# Show the resulting child xml metadata file example 
#for i in range(len(newMetaDataContent)):
print (newMetaDataContent)